In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Question Answering

## Overview

Recall the overall workflow for retrieval augmented generation (RAG):

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

Let's load our vectorDB.

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.5 MB/s eta 0:00:00


In [6]:
!pip install --quiet openai python-dotenv

In [10]:
!pip install --quiet langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [18]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 8.3 MB/s eta 0:00:00


In [33]:
#This is needed in order to for OpenAIEmbeddings. Please install it with `pip install tiktoken
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.7 MB/s eta 0:00:00


In [35]:
#Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version)
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 105.7 MB/s eta 0:00:00


In [24]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023.
LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [25]:
import os
import getpass

In [26]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [27]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


### load and segmentation of documents to create the db with FAISS (not persistent as in Chroma)

In [15]:
# persist_directory = "/docs/faiss/"

In [16]:
# !del .\docs\faiss\*

/bin/bash: line 1: del: command not found


In [28]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_001_NG_Dinarp_2023.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_002_NG_Dinarp_2023.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_003_NG_Dinarp_2023.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Capacitacion/DeepLearning.AI/data/Resolucion_no_008_NG_Dinarp_2023.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [29]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [30]:
splits = text_splitter.split_documents(docs)

In [31]:
len(splits)

102

In [36]:
embedding = OpenAIEmbeddings()

vectordb = FAISS.from_documents(
    splits,
    embedding
)

In [37]:
query = "¿Cuál es el nombre del Abg en la cabecera de la RESOLUCIÓN Nro. 008-NG-DINARP-2023?"
docs = vectordb.similarity_search(query)

In [38]:
len(docs)

4

In [39]:
print(docs[0].page_content)

Excepcionales de Datos e Información de las Entidades que Conforman el SINARDAP.  
 
La presente resolución entrará en vigencia a partir de la fecha de su suscripción, sin 
perjuicio de su publicación en el R egistro Oficial.  
 
Dado en la ciudad de Quito, a los 12 días del mes de mayo de 2023.  
 
 
 
 
 
 
 
Abg. Daniel Augusto Arboleda Villacreses  
DIRECTOR NACIONAL (E)  
DIRECCIÓN  NACIONAL DE REGISTROS PÚBLICOS


### Regresamos al notebook original

In [40]:
print(len(vectordb.index_to_docstore_id))

102


In [41]:
question = "¿Que manifiesta el artículo 83 de la Ley de Transformación Digital y Audiovisual?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [43]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [ ]:
#print(llm)

In [ ]:
#llm

### RetrievalQA chain

In [46]:
from langchain.chains import RetrievalQA

In [47]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [48]:
result = qa_chain({"query": question})

In [49]:
result["result"]

'El artículo 83 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro, manifiesta: “Agréguese a continuación del literal g) del artículo 11, el siguiente: h) Desarrollar e implementar los sistemas informáticos que permitan la transformación a formato digital de todos los registros, certificados, inventarios y demás actos o constancias físicas que genere el Registro para lo cual deberá tomar en cuenta las características y condiciones que para dicha finalidad estará condicionado a la mejora de los canales u otros componentes tecnológicos por parte del ente registral y la Dirección Nacional de Registros Públicos. Los registros digitales que se generen como consecuencia de la transformación a la que se refiere este literal, así como todos los registros digitales que existan al momento o que se generen en el futuro, deberán contar con todas las medidas de ciberseguridad necesarias para garantizar la seguridad de la información que reposa en ellos, así como su con

### Prompt

In [50]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Usa la siguiente pieza de contexto para responder la PREGUNTA al final. Si no sabes la respuesta, solo di que tu no sabes, no trates de inventar una respuesta. Usa tres frases máximo.  Mantén la respuesta lo más consisa posible. Siempre di "Gracias por preguntar", al final de la respuesta.
{context}
PREGUNTA: {question}
Respuesta:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [51]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [52]:
question = "¿Qué señala el artículo 86 de la Ley de Transformación Digital y Audiovisual?"

In [53]:
result = qa_chain({"query": question})

In [54]:
result["result"]

'El artículo 86 de la Ley de Transformación Digital y Audiovisual señala que los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica, adaptando los procedimientos contemplados en la ley para los registros físicos de los documentos. Gracias por preguntar.'

In [55]:
result["source_documents"][0]

Document(page_content='Los registros digitales que se generen como consecuencia de la transformación a \nla que se refiere este literal, así como todos l os registros digitales que existan al \nmomento o que se generen en el futuro, deberán contar con todas las medidas de \nciberseguridad necesarias para garantizar la seguridad de la información que \nreposa en ellos, así como su confidencialidad, integridad y di sponibilidad, \nincluyendo las garantías de protección de datos personales (…).”;  \nQue  el artículo 85 de la Ley de Transformación Digital y Audiovisual que reforma la \nLey de Registro, dispone: “ Agréguese en el artículo 18, el siguiente texto: “El \nLibro Reper torio podrá llevarse de forma electrónica en aquellos Registros que \nimplemente un proceso de digitalización y automatización de los trámites \nregistrales a su cargo. Para lo cual el proceso electrónico de registro en este Libro \ndebe llevarse con sujeción al  presente artículo.””;  \nQue  el artículo 86 de Ley 

### RetrievalQA chain types

In [56]:
# using map_reduce
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [57]:
result = qa_chain_mr({"query": question})

In [58]:
result["result"]

'No se menciona el artículo 86 de la Ley de Transformación Digital y Audiovisual en el texto proporcionado, por lo que no se puede determinar qué señala dicho artículo.'

If you wish to experiment on the `LangChain plus platform`:

 * Go to [langchain plus platform](https://www.langchain.plus/) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

In [ ]:
#import os
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
#os.environ["LANGCHAIN_API_KEY"] = "..." # replace dots with your api key

In [59]:
# map_reduce
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'No se menciona el artículo 86 de la Ley de Transformación Digital y Audiovisual en el texto proporcionado, por lo que no se puede determinar qué señala dicho artículo.'

In [61]:
# refine
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'El artículo 86 de la Ley de Transformación Digital y Audiovisual establece las características y condiciones que deben cumplir los sistemas informáticos para permitir la transformación a formato digital de todos los registros. Esto se basa en los resultados obtenidos durante el evento "Foro Nacional de Registros - Primera edición", donde se determinó el tiempo prudente para la transformación digital de los registros y la priorización al momento de digitalizar. Además, se menciona que el Director de Asesoría Jurídica de la Dirección Nacional de Registros Públicos, designado por el Ministerio de Telecomunicaciones y de la Sociedad de la Información, ejercerá las competencias y atribuciones previstas en la Ley Orgánica del Sistema Nacional de Registros Públicos y demás normativa aplicable. También se establece la obligación de las instituciones de la Función Ejecutiva de colaborar en la generación de los instrumentos necesarios para la aplicación de la Ley de Transformación Digital y uti

### RetrievalQA limitations

QA fails to preserve conversational history.

In [62]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [63]:
question = "¿Qué señala el artículo 86 de la Ley de Transformación Digital y Audiovisual?"
result = qa_chain({"query": question})
result["result"]

'El artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro señala: "Agréguese a continuación del artículo 24, en el TÍTULO V, el siguiente artículo innumerado: \'Art. (…). - Los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica, para lo cual se tomarán en consideración y serán adaptados todos los procedimientos contemplados en la presente Ley para los registros físicos de los documentos\'".'

In [64]:
question = "¿Cómo podrán llevarse los registros según el artículo 86?"
result = qa_chain({"query": question})
result["result"]

'Según el artículo 86 de la Ley de Transformación Digital y Audiovisual que reforma la Ley de Registro, los registros de las inscripciones y el libro de índice general podrán llevarse de forma electrónica. Para ello, se tomarán en consideración y se adaptarán todos los procedimientos contemplados en la presente Ley para los registros físicos de los documentos.'

Note, The LLM response varies. Some responses **do** include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.